In [ ]:
import csv


def get_lists_platforms_and_series():
    
    rows = []
        
    with open('geo_samples.tsv','r') as tsvin, open('series_vs_platforms.csv', 'w') as csvout:
        tsvin = csv.reader(tsvin, delimiter='\t')
        csvout = csv.writer(csvout, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['Series', 'Taxonomy', 'Platform']]
                rows.append(['Series', 'Platforms'])
            
            if row[indexes[1]].lower() in {"homo sapiens", "mus musculus", "rattus norvegicus"}:
                print(i)
                ser = row[indexes[0]].split(";")
                for s in ser: 
                    r = [s, row[indexes[2]]]
                    if r not in rows:
                        rows.append(r)
                        
        print("Length rows: ", len(rows))
        csvout.writerows(rows)
                        

get_lists_platforms_and_series()

In [13]:
import csv

arr = []
with open('table.csv', 'w') as csvout:
    csvout = csv.writer(csvout, delimiter='\t')
    arr.append(['e', 't'])
    arr.append(['r', 'yy'])
    w = ['e', 't']
    if w not in arr: 
        print("not")
        arr.append(w)
    # for a in arr:
    #     for key, value in a.items():
    #         print(key, " ", value)
    print(arr) 
    csvout.writerows(arr)


[['e', 't'], ['r', 'yy']]


In [28]:
import csv


def get_paths(file_name):
    
    paths = {}
    
    with open(file_name,'r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter=',')
        indexes = []
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['key', 'path']]
                
            paths[row[indexes[0]]] = row[indexes[1]]
            
    return paths



def get_platforms_quantity_for_each_serie():
    import collections
    
    series = []
    
    with open('series_vs_platforms.csv','r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['Series', 'Platforms']]
            
            series.append(row[indexes[0]])
    
    duplicated_series = \
            [item for item, count in collections.Counter(series).items() if count > 1]
    
    quantity = {}
    count_not_in_s = 0
    
    for s in series:
        if s not in quantity:
            quantity[s] = 'duplicate' if s in duplicated_series else 'single'
    
    return quantity



def get_full_file_with_urls_series_and_platforms(series_paths, platforms_paths, quantity):
    rows = []
        
    with open('series_vs_platforms.csv','r') as tsvin, \
                        open('urls_series_and_platforms_full_list.csv', 'w') as csvout:
            
        tsvin = csv.reader(tsvin, delimiter='\t')
        csvout = csv.writer(csvout, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in ['Series', 'Platforms']]
                rows.append(['Series', 'Series_url', 'Platforms', 'Platforms_url'])
            
            if row[indexes[0]] in series_paths:
                serie_url = 'https://ftp.ncbi.nlm.nih.gov/geo/series/' + \
                                        series_paths[row[indexes[0]]]

                if quantity[row[indexes[0]]] == 'duplicate':
                    serie = row[indexes[0]] + '-' + row[indexes[1]] + '_series_matrix.txt.gz'
                else:
                    serie = row[indexes[0]] + '_series_matrix.txt.gz'

                serie_url = serie_url + serie
                
                if row[indexes[1]] in platforms_paths:
                    platform_url = 'https://ftp.ncbi.nlm.nih.gov/geo/platforms/' + \
                                            platforms_paths[row[indexes[1]]] + row[indexes[1]] + \
                                            '.annot.gz'
                    rows.append([row[indexes[0]], serie_url, row[indexes[1]], platform_url])   
            
        csvout.writerows(rows)
        

In [ ]:
series_paths = get_paths('list_series.csv')
platforms_paths = get_paths('list_platforms.csv')
quantity = get_platforms_quantity_for_each_serie()
print('Strart fill url file')
get_full_file_with_urls_series_and_platforms(series_paths, platforms_paths, quantity)

In [39]:
import csv


def get_full_list_urls_separate(file_name, kind_data):
    rows = []
        
    with open('urls_series_and_platforms_full_list.csv','r') as tsvin, \
                        open(file_name, 'w') as csvout:
            
        tsvin = csv.reader(tsvin, delimiter='\t')
        csvout = csv.writer(csvout, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                indexes = [row.index(j) for j in \
                           ['Series', 'Series_url', 'Platforms', 'Platforms_url']]
                if kind_data == 'series':
                    rows.append(['Series', 'Series_url'])
                else:
                    rows.append(['Platforms', 'Platforms_url'])
            else:
                r = []
                if kind_data == 'series':
                    r = [row[indexes[0]], row[indexes[1]]]
                else:
                    r = [row[indexes[2]], row[indexes[3]]]
                rows.append(r)     

        csvout.writerows(rows)


get_full_list_urls_separate('full_list_urls_series.csv', 'series')
print('Start fill platforms')
get_full_list_urls_separate('full_list_urls_platforms.csv', 'platforms')

Start fill platforms


In [73]:
import csv


def get_short_list_urls_separate(file_name_for_reading, file_name_for_writing,\
                                file_name_with_current_data, kind_data):
    rows = []
    current_data = format_file(file_name_with_current_data)
        
    with open(file_name_for_reading,'r') as tsvin, \
                        open(file_name_for_writing, 'w') as csvout:
            
        tsvin = csv.reader(tsvin, delimiter='\t')
        csvout = csv.writer(csvout, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                if kind_data == 'series':
                    indexes = [row.index(j) for j in ['Series', 'Series_url']]
                    rows.append(['Series', 'Series_url'])
                else:
                    indexes = [row.index(j) for j in ['Platforms', 'Platforms_url']]
                    rows.append(['Platforms', 'Platforms_url'])
            else: 
                dif_string = 'https://ftp.ncbi.nlm.nih.gov/geo/' + kind_data + '/'
                path = row[indexes[1]].replace(dif_string, '')
                               
                if path not in current_data:
                    rows.append([row[indexes[0]], row[indexes[1]]])     

        csvout.writerows(rows)



def format_file(file_name):
    lines = [line.rstrip('\n') for line in open(file_name)]
    for i in range(0,len(lines),1):
        lines[i] = lines[i].replace('./', '')
    return lines


get_short_list_urls_separate('full_list_urls_series.csv', \
                'short_list_urls_series.csv', 'current_series.txt', 'series')
                               
# print('Start fill platforms')
                               
# get_short_list_urls_separate('full_list_urls_platforms.csv', \
#                 'short_list_urls_platforms.csv', 'current_platforms.txt', 'platforms')


In [ ]:
import subprocess
import csv


def download_data(file_name, kind_data): 
    
    with open(file_name,'r') as tsvin:
        tsvin = csv.reader(tsvin, delimiter='\t')
        indexes = []
        
        for (i, row) in enumerate(tsvin):
            if i == 0:
                if kind_data == 'series':
                    indexes = [row.index(j) for j in ['Series', 'Series_url']]
                else:
                    indexes = [row.index(j) for j in ['Platforms', 'Platforms_url']]
            else:    
                try:
                    dif_string = 'https://ftp.ncbi.nlm.nih.gov/'
                    path = row[indexes[1]].replace(dif_string, '')
                    path = path[:(path.rindex('/')+1)]
                    directory = "../" + path

                    subprocess.call(["mkdir", "-p", directory])
                    subprocess.call(["wget", "-c", "-P", directory, row[indexes[1]]])

                except:
                    pass

            print('current', kind_data, ': ', row[indexes[0]], 'url: ', \
                                          row[indexes[1]], ' with i: ', i)

        
        
download_data("short_list_urls_platforms.csv", 'platforms')
#download_data('short_list_urls_series.csv', 'series')

In [72]:
lines = [line.rstrip('\n') for line in open("current_series.txt")]
len(lines)

42474

In [66]:
s = "https://ftp.ncbi.nlm.nih.gov/geo/platforms/GPLnnn/GPL570/annot/GPL570.annot.gz"
dif_string = 'https://ftp.ncbi.nlm.nih.gov/'
path = s.replace(dif_string, '')
path[:(path.rindex('/')+1)]

'geo/platforms/GPLnnn/GPL570/annot/'